In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# DCGAN= Deep Convolution GANs
Upsampling convolution
It is also called often.
`Deconvolution or 
Fractionally strided convolution or
Upconvolution`

# Output Size= (I-1)*S-2P+K
I=Input size
S=Stride
P=Padding
K=Kernel


In [2]:
import torch
import torch.nn as nn

input=torch.randn(1,1,6,6)
input

model=nn.Sequential(
    nn.Conv2d(in_channels=1,
             out_channels=10,
             kernel_size=2,
             padding=1,
             stride=1,
             bias=True),
    nn.LeakyReLU(0.3),
    nn.BatchNorm2d(10),
    nn.Dropout(0.3),
    nn.Tanh()
    
)

output=model(input)
output.shape
# output

torch.Size([1, 10, 7, 7])

In [3]:
import torch 
import torch.nn as nn

input= torch.randn(1,3,8,8)
input

model=nn.Sequential(
    nn.ConvTranspose2d(
        in_channels=3,
        out_channels=1,
        kernel_size=2,
             padding=1,
             stride=1,
             bias=True
        
    ),
    nn.LeakyReLU(0.3),
    nn.BatchNorm2d(1),
    nn.Dropout(0.3),
    nn.Tanh()
)

output=model(input)
output.shape

output


tensor([[[[ 0.0000, -0.6504, -0.0000, -0.5408, -0.1258, -0.6539,  0.9604],
          [ 0.9874, -0.9042, -0.9640, -0.0000, -0.6443, -0.7513, -0.5277],
          [ 0.0000, -0.0000, -0.9891, -0.0000, -0.2398, -0.0000,  0.8462],
          [-0.3257, -0.9101, -0.7892,  0.8604, -0.0000, -0.3516,  0.0634],
          [ 0.6609, -0.0000, -0.3346,  0.9805, -0.8764,  0.9658, -0.2578],
          [ 0.7720, -0.4950, -0.7633, -0.0000,  0.9680,  0.9994, -0.0000],
          [-0.3363, -0.0000,  0.8613,  0.9360,  0.6868,  0.6365,  0.0000]]]],
       grad_fn=<TanhBackward0>)

# f(x)=x if x≥0
#      αx if x<0

In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
import os

latent_dim = 100
img_size = 32
channels = 3
batch_size = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 50
lr = 0.0002
beta1 = 0.5

os.makedirs("dcgan_cifar10_output", exist_ok=True)

transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  # Pixel: [-1, 1]
])

dataloader = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR10(root="./data", download=True, transform=transform),
    batch_size=batch_size,
    shuffle=True
)


100%|██████████| 170M/170M [00:11<00:00, 15.3MB/s]


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os

latent_dim = 100
img_channels = 3
img_size = 32
batch_size = 128
lr = 0.0002
beta1 = 0.5
epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

os.makedirs("dcgan_outputs", exist_ok=True)

transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model=nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0), #in_chan,out_chan, kernel, stride, padd
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.ConvTranspose2d(512, 256, 4, 2,1), #in_chan,out_chan, kernel, stride, padd
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.ConvTranspose2d(256, 128, 4, 2, 1),       # (B,128,16,16)
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, img_channels, 4, 2, 1),# (B,3,32,32)
            nn.Tanh()
        )
    def forward(self,z):
        return self.model(z)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model= nn.Sequential(
            nn.Conv2d(img_channels, 128, 4,2,1), #[B,128, 16, 16]
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128,256,4,2,1),   #8*8
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2,inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1),  #4*4
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512,1,2,2,0),
            nn.Sigmoid()

            
        )
    def forward(self, img):
        out=self.model(img)
        return out.view(-1,1).squeeze(1)
        

model_Gen= Generator().to(device)
model_Gen

model_Dis=Discriminator().to(device)

model_Dis


Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(512, 1, kernel_size=(2, 2), stride=(2, 2))
    (9): Sigmoid()
  )
)

In [7]:
loss_fn=nn.BCELoss()

optimizer_G=torch.optim.Adam(params=model_Gen.parameters(),lr=lr, betas=(beta1, 0.999))
optimizer_D=torch.optim.Adam(params=model_Dis.parameters(),lr=lr, betas=(beta1, 0.999))


fixed_noise=torch.randn(64, latent_dim, 1, 1, device=device)


